Note that, differently from a linear problem (e.g., the problems presented in the notebooks [least_squares](https://nbviewer.jupyter.org/github/birocoles/Disciplina-metodos-computacionais/blob/master/Content/least_squares.ipynb) and [grav_net](https://nbviewer.jupyter.org/github/birocoles/Disciplina-metodos-computacionais/blob/master/Content/grav_net.ipynb)), the derivative of the predicted data $\Delta t_{i} (\mathbf{p})$, $i = A, B, C$, with respect to $x_{0}$ or $y_{0}$ is still a function of $x_{0}$ and $y_{0}$. Consequently, setting the gradient of $\Phi(\mathbf{p})$ to the null vector does not result in a linear system for estimating $\mathbf{p}^{\ast}$. The problem of estimating $\mathbf{p}^{\ast}$ is an example of non-linear optimization problem and needs to be solved iteratively.

## Synthetic data

<a id='Auxiliary functions'></a>
#### Auxiliary functions

In [ ]:
def predicted_data(vp,vs,x,y,p):
    '''
    Computes the predicted data.
    
    input
    vp, vs: floats - seismic velocities
    x, y: numpy arrays - coordinates x and y of the stations
    p: numpy array - parameter vector
    
    output
    dt: numpy array - predicted data
    '''
    
    if vp <= vs:
        raise ValueError('vp must be greater than vs')
    
    if x.size != y.size:
        raise ValueError('x and y must have the same number of elements')
    
    if (vp != 0.) & (vs != 0.):
        alfa = 1./vs - 1./vp
    else:
        raise ValueError('vp and vs must be non-null')
    
    # predicted data
    dt = alfa*np.sqrt((x - p[0])**2 + (y - p[1])**2)
    
    return dt

In [ ]:
def plot_data(data, filename=None):
    '''
    input
    data: list of numpy arrays - data sets to be plotted.
          In the case of two data sets, the first 
          one must be the observed data.
    filename: string or None - if not None, it saves the figure with
              name defined by filename.
   
    output
    matplotlib figure
    '''

    assert (len(data) == 1) or (len(data) == 2), \
'must use one or two data sets'

    plt.figure(figsize=(8,3))
    stations_label = ['A', 'B', 'C']
    stations = [1,2,3]
    
    if len(data) == 1:
        plt.barh(stations, data[0], align='center', 
             color='r', alpha=1, label='Observed')
    if len(data) == 2:
        plt.barh(stations, data[0], align='center', 
                 color='r', alpha=0.6, label='Observed')
        plt.barh(stations, data[1], align='center', 
                 fill=False, edgecolor='b', alpha=1.,
                 linewidth=3., linestyle='dashed', label='Predicted')
    plt.yticks(stations, stations_label, fontsize=18)
    plt.xticks(fontsize=14)
    plt.xlabel('$\Delta t$ ($s$)', fontsize=18)
    plt.ylabel('Stations', fontsize=16)
    plt.grid()
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), 
               loc=3, ncol=2, borderaxespad=0.)
    if filename is not None:
        plt.savefig(filename)
    plt.show()

In [ ]:
def goal_function_grid(dobs,vp,vs,x,y,x0,y0):
    '''
    Computes the goal function in a grid of
    parameters x0 and y0.
    
    input
    dobs: numpy array - observed data
    vp, vs: floats - seismic velocities
    x, y: numpy arrays - coordinates x and y of the stations
    
    output
    phi: numpy array - grid of goal function values
    '''
    
    if vp <= vs:
        raise ValueError('vp must be greater than vs')
    
    if x.size != y.size:
        raise ValueError('x and y must have the same number of elements')
    
    if (vp != 0.) & (vs != 0.):
        alfa = 1./vs - 1./vp
    else:
        raise ValueError('vp and vs must be non-null')
   
    phi = np.zeros_like(x0)
    for x, y, dt in zip(x, y, dobs):
        phi += (dt - alfa*np.sqrt((x - x0)**2 + (y - y0)**2))**2
    return phi

In [ ]:
def convergence(phi0,phi,beta):
    '''
    Evaluate the convergence criterion.
    '''

    # relative variation
    rel_variation = abs((phi0-phi)/phi0)
    
    return rel_variation < beta

In [ ]:
def plot_goal_function(x0, y0, phi, p_true=None, stations=None, epicenters=None, 
                       filename=None):
    '''
    Plot the goal function, the stations, the true epicenter
    location and the set of estimated epicenters produced by
    an iterative method.
    
    input
    x0, y0: numpy arrays 2D - grid of epicenter coordinates
            on which the goal function is evaluated
    phi: numpy array 2D - grid of goal function values
    p_true: None or numpy array 1D containing the true coordinates
            of the epicenter
    stations: numpy array 2D - matrix containing the coordinates of
              the stations. The first collumn contains the
                x coordinate and the second collumn contains the y coordinate
                of the stations.
    epicenters: numpy array 2D - matrix containing the estimated coordinates
                computed by an iterative method. The first collumn contains the
                x coordinate and the second collumn contains the y coordinate
                of the estimated epicenters.
    filename: string or None - if not None, it saves the figure with
              name defined by filename
    
    output
    matplotlib figure
    '''
    
    if p_true is not None:
        assert p_true.size == 2, 'p_true must have 2 elements'
    if stations is not None:
        assert (stations.shape[1] == 2), 'stations must have 2 collumns'
        assert (stations.shape[0] >= 3), \
'the number of stations must be greater than or equal to 3'

    xmax = np.max(x0)
    xmin = np.min(x0)
    
    ymax = np.max(y0)
    ymin = np.min(y0)
    
    plt.figure(figsize=(8,6))
    plt.axis('scaled')
    plt.contourf(y0,x0,phi,40, cmap = plt.get_cmap('viridis'))
    cbar = plt.colorbar()
    cbar.set_label('$\Delta t$ ($s$)', fontsize=16)
    if stations is not None:
        plt.plot(stations[:,1], stations[:,0],'k^', markersize=14)
    if p_true is not None:
        plt.plot([p_true[1]], [p_true[0]], 'ro', markersize=14)
    if epicenters is not None:
        plt.plot(epicenters[:,1],epicenters[:,0],'w-', linewidth=1)
        plt.plot(epicenters[:,1],epicenters[:,0],'wo', markersize=6)
    plt.xlabel('y (km)', fontsize=14)
    plt.ylabel('x (km)', fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlim(ymin,ymax)
    plt.ylim(xmin,xmax)
    if filename is not None:
        plt.savefig(filename)
    plt.show()

In [ ]:
def plot_phi(phi, filename=None):
    '''
    Plot the goal function values along the
    iterations.
    
    input
    phi: list - goal function values
    filename: string or None - if not None, it saves the figure with
              name defined by filename
    
    output
    matplotlib figure
    '''
    
    it = [i for i in range(len(phi))]
    plt.figure(figsize=(8,8))
    plt.plot(it,phi,'ko')
    plt.plot(it,phi,'k-')
    plt.xlabel('$k$', fontsize=18)
    plt.ylabel('$\phi(\mathbf{p})$', fontsize=18)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.grid()
    plt.xlim(0, len(phi))
    plt.ylim(-5.,phi[0]+5.)
    if filename is not None:
        plt.savefig(filename)
    plt.show()

#### Import the required packages

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

#### Limits of the simulated study area

In [ ]:
# limits of the study area in km
xmin = 0.
xmax = 100.
ymin = -50.
ymax = 50.

#### Seismic velocities

In [ ]:
# seismic velocities in km/s
vp = 6.5
vs = 6.5/1.6

#### Coordinates of the epicenter

In [ ]:
# coordinates of the simulated epicenter in km
p_true = np.array([50., -17.])

#### Coordinates of the seismic stations

In [ ]:
# coordinates of the seismic stations in km
x = np.array([10., 80., 60.])

y = np.array([-40., -20., 35.])

stations = np.vstack([x, y]).T

In [ ]:
plt.figure(figsize=(8,6))
plt.axis('scaled')
plt.plot(y, x,'k^', markersize=14, label='Stations')
plt.plot([p_true[1]], [p_true[0]], 'ro', markersize=14, label='True epicenter')
plt.xlabel('y (km)', fontsize=14)
plt.ylabel('x (km)', fontsize=14)
plt.legend(loc='best', numpoints=1)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlim(ymin,ymax)
plt.ylim(xmin,xmax)
plt.show()

#### Synthetic data vector

Given the model defined in the previous cells, we can simulate an observed data vector $\mathbf{d}$ by using the function `predicted_data` as follows:

In [ ]:
d = predicted_data(vp, vs, x, y, p_true)

#### Goal function

In this example, the goal function depends on two parameters, which represent the Cartesian coordinates of the epicenter. It means that, given a point $(x_{0},y_{0})$, and consequently a parameter vector $\mathbf{p}$, we can calculate the function $\Phi(\mathbf{p})$. Then, if we define a grid of points $(x_{0},y_{0})$, each one representing a possible epicenter, we can calculate the goal function $\Phi(\mathbf{p})$ on a grid.

The grid of points representing possible epicenter can be defined as follows: